# Cuda test

In [ ]:
!nvidia-smi

In [ ]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
import torch

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
	
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")


# load pdf

## Exemplo com UnstructuredPDFLoader

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

In [ ]:
local_path = r"data\WEF_The_Global_Cooperation_Barometer_2024.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [ ]:
# Preview first page
print(Markdown(data[0].page_content))

## Exemplo com docling

In [ ]:
from docling.document_converter import DocumentConverter

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import EasyOcrOptions, PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

local_path = r"data\WEF_The_Global_Cooperation_Barometer_2024.pdf"

#artifacts_path = "C:/Users/emerson/.cache/docling/models"
# pipeline_options = PdfPipelineOptions(artifacts_path=artifacts_path)
pipeline_options = PdfPipelineOptions()
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)
result = converter.convert(local_path)
print(result.document.export_to_markdown())  # output: "### Docling Technical Report[...]"

# Vector Embeddings

## olama

### testing ollama

In [ ]:
!ollama list

In [ ]:
# # Pull nomic-embed-text model from Ollama if you don't have it
# !ollama pull nomic-embed-text
# # List models again to confirm it's available
# !ollama list

In [ ]:
# # 1. First clean up any existing ChromaDB installations
# %pip uninstall -y chromadb
# %pip uninstall -y protobuf

# # 2. Install specific versions known to work together
# %pip install -q protobuf==3.20.3
# %pip install -q chromadb==0.4.22  # Using a stable older version
# %pip install -q langchain-ollama

### text split

In [ ]:
# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

#### UnstructuredPDFLoader

In [ ]:
print(len(data))
print(data[0])

In [ ]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [ ]:
print(len(chunks))
print(chunks[0])

#### docling

In [ ]:
from docling.document_converter import DocumentConverter

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import EasyOcrOptions, PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

local_path = r"data\WEF_The_Global_Cooperation_Barometer_2024.pdf"

#artifacts_path = "C:/Users/emerson/.cache/docling/models"
# pipeline_options = PdfPipelineOptions(artifacts_path=artifacts_path)
pipeline_options = PdfPipelineOptions()
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)
result = converter.convert(local_path)
print(result.document.export_to_markdown())  # output: "### Docling Technical Report[...]"

In [ ]:
# display(result.document.export_to_markdown())
from langchain_core.documents import Document

documento = Document(page_content=result.document.export_to_markdown(), metadata={"source": local_path})

print(documento)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents([documento])

In [ ]:
print(len(chunks))
print(chunks[0])

Obs: precisa exceutar o ollama aqui

## Indexar Chromadb

In [ ]:
import hashlib
import os
import chromadb
from chromadb.config import Settings
from langchain.docstore.document import Document
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings 

def generate_id(document, page_index):
    """Gera um ID único baseado no nome do arquivo e no índice da página."""
    source = document.metadata['source']
    filename = os.path.basename(source)
    base_id = hashlib.sha256(filename.encode()).hexdigest()
    return f"{base_id}_{page_index}"

def check_and_add_document(collection, document, page_index, embedding_model):
    """Verifica se um documento já existe na coleção e o adiciona se não existir."""
    document_id = generate_id(document, page_index)

    # Verifica se o ID já existe na coleção
    results = collection.get(ids=[document_id])
    if results['ids'] and document_id in results['ids']:
        print(f"Documento com ID {document_id} já existe na coleção.")
        return

    # Adiciona o documento à coleção
    embedding = embedding_model.embed_documents([document.page_content])[0]
    collection.add(documents=[document.page_content], ids=[document_id], embeddings=[embedding], metadatas=[document.metadata])
    print(f"Documento com ID {document_id} adicionado à coleção.")

# exemplo inline de chunks
# chunks = [
#     Document(page_content="This is a document about pineapple", metadata={"source": "file1.txt"}),
#     Document(page_content="This is a document about oranges", metadata={"source": "file2.txt"}),
#     Document(page_content="Another document about pineapple", metadata={"source": "file1.txt"}),
#     Document(page_content="New document about oranges", metadata={"source": "file2.txt"}),
# ]

collection_name = "local-rag"
embedding_model = OllamaEmbeddings(model="nomic-embed-text")
persist_directory = "chroma/chroma_db"  # Diretório onde o banco de dados será salvo

# Inicializa o cliente ChromaDB
chroma_client = chromadb.PersistentClient(path=persist_directory, settings=Settings(allow_reset=False))

# Obtém ou cria a coleção
collection = chroma_client.get_or_create_collection(name=collection_name)
print("Coleção carregada.")

# Itera sobre os chunks e os adiciona à coleção, verificando se já existem
for i, chunk in enumerate(chunks):
    check_and_add_document(collection, chunk, i, embedding_model)

# Exemplo para verificar o ID do primeiro chunk
if chunks:
    first_chunk_id = generate_id(chunks[0], 0)
    print(f"ID do primeiro chunk: {first_chunk_id}")

# # Exemplo de query para verificar se os dados foram adicionados corretamente
# query_embedding = embedding_model.embed_query("fruit information")
# query_results = collection.query(query_embeddings=[query_embedding], n_results=2)
# print("\nResults of query:")
# for result in query_results['documents'][0]:
#     print(result)

# TESTE Chroma

In [ ]:
import chromadb
from chromadb.config import Settings
chroma_client = chromadb.PersistentClient(path=r"chroma\testes", settings=Settings(allow_reset=True))
chroma_client.reset()

chroma_client = chromadb.Client()

collection = chroma_client.get_or_create_collection(name="teste-rag")

collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

# End teste

In [ ]:
results = collection.get(limit=5)
display(results)

# # Os resultados são um dicionário; você precisa processá-los para obter os documentos
# ids = results['ids']
# documents = results['documents']
# metadatas = results['metadatas']

# for i in range(len(ids)):
#     print(f"ID: {ids[i]}")
#     print(f"Conteúdo: {documents[i]}")
#     print(f"Metadados: {metadatas[i]}")
#     print("-" * 50)

# Retrieval

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "llama3.2"
local_model = "deepseek-r1"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar cinco
    versões diferentes da pergunta do usuário fornecida para recuperar documentos relevantes de
    um banco de dados vetorial. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu
    objetivo é ajudar o usuário a superar algumas das limitações da pesquisa de similaridade 
    baseada em distância. Forneça essas perguntas alternativas separadas por quebras de linha.
    Pergunta original: {question}""",
)

In [ ]:
from langchain_ollama import OllamaEmbeddings 
from langchain_chroma import Chroma

embedding_model = OllamaEmbeddings(model="nomic-embed-text")

#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
vector_db = Chroma(
    client=chroma_client,
    collection_name=collection_name,
    embedding_function=embedding_model
)

retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Responda à pergunta com base SOMENTE no seguinte contexto:
{context}
Pergunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(chain.invoke("Quais são os 5 pilares da cooperação global?"))

# Excluir Coleção

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()
print(f"Coleção '{collection_name}' deletada.")